# Attempt to generate an `AlchemicalState` and a `CompoundThermodynamicState` with which to manipulate $ \lambda $ parameters 

In [56]:
from openmmtools import testsystems
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion
from openmmtools.states import CompoundThermodynamicState, SamplerState, ThermodynamicState
from simtk import unit

In [57]:
ala_vac_sys = testsystems.AlanineDipeptideVacuum().system
factory = AbsoluteAlchemicalFactory(consistent_exceptions = False)
alchemical_region = AlchemicalRegion(alchemical_atoms=range(22))
alanine_alchemical_system = factory.create_alchemical_system(reference_system=ala_vac_sys,
                                                            alchemical_regions = alchemical_region)

In [58]:
first_alchemical_state = AlchemicalState.from_system(alanine_alchemical_system)

In [59]:
vars(first_alchemical_state)

{'_function_variables': {},
 '_parameters': {'lambda_bonds': None,
  'lambda_electrostatics': 1.0,
  'lambda_sterics': 1.0,
  'lambda_angles': None,
  'lambda_torsions': None},
 '_parameters_name_suffix': None}

In [60]:
thermodynamic_state = ThermodynamicState(system=alanine_alchemical_system,
                                         temperature=300*unit.kelvin)

In [61]:
comp = CompoundThermodynamicState(thermodynamic_state, composable_states = [first_alchemical_state])

In [62]:
comp._composable_states[0]._parameters


{'lambda_bonds': None,
 'lambda_electrostatics': 1.0,
 'lambda_sterics': 1.0,
 'lambda_angles': None,
 'lambda_torsions': None}

In [63]:
comp.lambda_sterics = .1



In [64]:
comp._composable_states[0]._parameters

{'lambda_bonds': None,
 'lambda_electrostatics': 1.0,
 'lambda_sterics': 0.1,
 'lambda_angles': None,
 'lambda_torsions': None}

In [69]:
sampler_state = SamplerState(testsystems.AlanineDipeptideVacuum().positions)

## let's make an integrator

In [81]:
from openmmtools.mcmc import LangevinSplittingDynamicsMove as langevin
langevin_move = langevin(timestep = 1. * unit.femtoseconds, collision_rate = 1./unit.picoseconds, n_steps = 1, reassign_velocities = True, measure_heat = True)


In [82]:
langevin_move.apply(comp, sampler_state)

In [87]:
dir(langevin_move)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_after_integration',
 '_before_integration',
 '_get_integrator',
 'apply',
 'collision_rate',
 'constraint_tolerance',
 'context_cache',
 'measure_heat',
 'measure_shadow_work',
 'n_restart_attempts',
 'n_steps',
 'reassign_velocities',
 'splitting',
 'timestep']

In [84]:
langevin_move._get_integrator(comp).heat

Quantity(value=0.0, unit=kilojoule/mole)